# Step 3: Perform fit

In [ ]:
%reset -f
import pycompwa.ui as pwa
particle_list = pwa.read_particles('model.xml')
kinematics = pwa.create_helicity_kinematics('model.xml', particle_list)
kinematics.create_all_subsystems()
data_sample = pwa.read_root_data(input_file='generated_data.root')
phsp_sample = pwa.read_root_data(input_file='generated_phsp.root')
intensity_builder = pwa.IntensityBuilderXML(
    'model.xml', particle_list, kinematics, phsp_sample)
intensity = intensity_builder.create_intensity()

An **Intensity** object behaves just like a mathematical function that takes a `DataSet` as an argument and returns a list of intensities (real numbers).

To perform a fit, first create an estimator of your choice. An estimator generally needs:

- an `Intensity` instance
- a `DataSet` (to which the intensity is fitted)
- optionally: a phase space `DataSet` (which is used to normalize the `Intensity`)

A phase space sample can be generated via the `generate_phsp()` function (see above). Since our `Intensity` is already normalized, this is not needed here.
The data samples can be converted to a `DataSet` as follows:

In [ ]:
data_set = kinematics.convert(data_sample)

Now we can create an **estimator**. Remember that an estimator indicates how well a set of model parameters describes a given data set best. In this example, we use an unbinned log likelihood estimator. The fit calculations are then performed the calculations with the `FunctionTree` back-end.

In [ ]:
estimator, initial_parameters = \
    pwa.create_unbinned_log_likelihood_function_tree_estimator(
        intensity, data_set)

Notice that you not only receive a estimator object, but also a list of fit parameters (`FitParameterList`). You use this list of fit parameters to initialize the optimizer later on. They contain the following info:

- the initial values of the parameters
- fix parameters
- define boundaries
- define errors, which can give certain optimizers hints on the step size

These fit parameters are initialized with the values stated in the XML file or with default values if unspecified. But can be changed easily in Python as well, like fixing certain parameters.

In [ ]:
from pprint import pprint
pprint(initial_parameters)
print("\nthis parameter is initially not fixed:")
print(initial_parameters[8])
initial_parameters[7].is_fixed = True
initial_parameters[8].is_fixed = True
print("\nand now it is fixed:")
print(initial_parameters[8])

To make the fit a bit more interesting, we modify one of the parameters to a different initial value than the true value.

In [ ]:
print("before:")
print(initial_parameters[12])
initial_parameters[12].value = 2.0
print("after:")
print(initial_parameters[12])

Now it's time to start up a set up a fit. This is quite simple: just create an optimizer instance of your choice, here Minuit2 (`MinuitIF`), and call its `optimize()` method to start the fitting process.

In [ ]:
optimizer = pwa.MinuitIF()
fit_result = optimizer.optimize(estimator, initial_parameters)

In [ ]:
fit_result.fit_duration_in_seconds

Let's check if the fit parameters are "close to" the true values.

In [ ]:
print("this should be close to 1.0 again:")
print(fit_result.final_parameters[12].value,
      "+-",
      fit_result.final_parameters[12].error)

**Important**: Note that the intensity instance still needs to be notified about this optimal set of parameters. They can be applied with the `updateParametersFrom()` function.

In [ ]:
intensity.updateParametersFrom(fit_result.final_parameters)

Again, you can store this fit result to disk and pick it up again:

In [ ]:
fit_result.write('fit_result.xml')
imported_fit_result = pwa.load('fit_result.xml')
imported_fit_result.final_parameters[12]